In [10]:
!pip install opencv-contrib-python==3.4.2.16

  Using cached https://files.pythonhosted.org/packages/b8/26/9c631442c828ec88d4b0017ec39bb6ed655d81b3e99010ad1c841d104d54/opencv_contrib_python-3.4.2.16-cp36-cp36m-macosx_10_6_x86_64.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl
  Found existing installation: opencv-contrib-python 3.2.0.7
    Uninstalling opencv-contrib-python-3.2.0.7:
      Successfully uninstalled opencv-contrib-python-3.2.0.7


In [2]:
!pip install imutils

  Created wheel for imutils: filename=imutils-0.5.3-cp36-none-any.whl size=25851 sha256=affd62f4af06ecfc102f31873f6a438d51d31544302f7f9de0a031945dafc375
  Stored in directory: /Users/judytraj/Library/Caches/pip/wheels/16/84/1f/bf88641293cda2c8be81a5c4b8ca973dd9125a6dc3767417fd
Successfully built imutils


In [1]:
import imutils
import cv2
import numpy as np

In [2]:
class Stitcher:
    def __init__(self):
        self.isv3 = imutils.is_cv3(or_better=True)
        
    def stitch(self, images, ratio=0.75, reprojThresh=4.0,
        showMatches=False):
        # unpack the images, then detect keypoints and extract local invariant descriptors from them
        (imageB, imageA) = images
        (kpsA, featuresA) = self.detectAndDescribe(imageA)
        (kpsB, featuresB) = self.detectAndDescribe(imageB)

        # match features between the two images
        M = self.matchKeypoints(kpsA, kpsB,
            featuresA, featuresB, ratio, reprojThresh)

        # if the match is None, then there aren't enough matched keypoints to create a panorama
        if M is None:
            return None
        # otherwise, apply a perspective warp to stitch the images together
        (matches, H, status) = M
        result = cv2.warpPerspective(imageA, H,
            (imageA.shape[1] + imageB.shape[1], imageA.shape[0]))
        result[0:imageB.shape[0], 0:imageB.shape[1]] = imageB

        # check to see if the keypoint matches should be visualized
        if showMatches:
            vis = self.drawMatches(imageA, imageB, kpsA, kpsB, matches,
                status)

            # return a tuple of the stitched image and the visualization
            return (result, vis)

        # return the stitched image
        return result
    
    def detectAndDescribe(self, image):
        # convert the image to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # detect and extract features from the image
        descriptor = cv2.xfeatures2d.SIFT_create()
        (kps, features) = descriptor.detectAndCompute(image, None)

        # convert the keypoints from KeyPoint objects to NumPy arrays
        kps = np.float32([kp.pt for kp in kps])

        # return a tuple of keypoints and features
        return (kps, features)
    
    def matchKeypoints(self, kpsA, kpsB, featuresA, featuresB,
        ratio, reprojThresh):
        # compute the raw matches and initialize the list of actual matches
        matcher = cv2.DescriptorMatcher_create("BruteForce")
        rawMatches = matcher.knnMatch(featuresA, featuresB, 2)
        matches = []

        # loop over the raw matches
        for m in rawMatches:
            # ensure the distance is within a certain ratio of each
            # other (i.e. Lowe's ratio test)
            if len(m) == 2 and m[0].distance < m[1].distance * ratio:
                matches.append((m[0].trainIdx, m[0].queryIdx))
                
        # computing a homography requires at least 4 matches
        if len(matches) > 4:
            # construct the two sets of points
            ptsA = np.float32([kpsA[i] for (_, i) in matches])
            ptsB = np.float32([kpsB[i] for (i, _) in matches])

            # compute the homography between the two sets of points
            (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC,
                reprojThresh)

            # return the matches along with the homograpy matrix and status of each matched point
            return (matches, H, status)

        # otherwise, no homograpy could be computed
        return None
    
    def drawMatches(self, imageA, imageB, kpsA, kpsB, matches, status):
        # initialize the output visualization image
        (hA, wA) = imageA.shape[:2]
        (hB, wB) = imageB.shape[:2]
        vis = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
        vis[0:hA, 0:wA] = imageA
        vis[0:hB, wA:] = imageB

        # loop over the matches
        for ((trainIdx, queryIdx), s) in zip(matches, status):
            # only process the match if the keypoint was successfully matched
            if s == 1:
                # draw the match
                ptA = (int(kpsA[queryIdx][0]), int(kpsA[queryIdx][1]))
                ptB = (int(kpsB[trainIdx][0]) + wA, int(kpsB[trainIdx][1]))
                cv2.line(vis, ptA, ptB, (0, 255, 0), 1)

        # return the visualization
        return vis

In [3]:
# Opens the Video file
cap= cv2.VideoCapture('Video.mp4')

In [7]:
count=1
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    cv2.imwrite('img'+str(count)+'.jpg',frame)
    count+=1

In [5]:
cap.release()
cv2.destroyAllWindows()

In [4]:
stitcher = Stitcher()

In [34]:
imgA = cv2.imread('img2.jpg')
imgB = cv2.imread('img1.jpg')
imageA = imutils.resize(imgA, width=400)
imageB = imutils.resize(imgB, width=400)

In [35]:
(result, vis) = stitcher.stitch([imageA, imageB], showMatches=True)
cv2.imwrite('out.jpg', result)

True

In [36]:
i = 5
while(i<count):
    imgA = cv2.imread('img'+str(i)+'.jpg')
    imgB = cv2.imread('out.jpg')
    (result, vis) = stitcher.stitch([imgA, imgB], showMatches=True)
    cv2.imwrite('out.jpg', result)
    i+=5